In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests

def augmented_dickey_fuller_test(df, column_name):
    print("ADF Test for: ", column_name)
    result = adfuller(df[column_name])
    print(f'Test Statistics: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'critical_values: {result[4]}')
    if result[1] > 0.05:
        print("Series is not stationary")
        return False
    else:
        print("Series is stationary")
        return True

test_result_1 = augmented_dickey_fuller_test(df, "stringency_index")
test_result_2 = augmented_dickey_fuller_test(df, " Gross Domestic Product (GDP)  Normalised")

if ((test_result_1 and test_result_2) == 0):
    df_transformed = pd.DataFrame()
    df_transformed["stringency_index"] = np.diff(np.log(df["stringency_index"]))
    df_transformed[" Gross Domestic Product (GDP)  Normalised"] = np.diff(np.log(df[" Gross Domestic Product (GDP)  Normalised"]))
    test_result_1 = augmented_dickey_fuller_test(df_transformed, "stringency_index")
    test_result_2 = augmented_dickey_fuller_test(df_transformed, " Gross Domestic Product (GDP)  Normalised")
    df = df_transformed
else:
    print("Both series are stationary...")

https://www.analyticsvidhya.com/blog/2021/08/vector-autoregressive-model-in-python/

https://www.statology.org/negative-aic/

https://www.statology.org/granger-causality-test-in-python/

In [ ]:
from statsmodels.tsa.api import VAR 

data = df[["stringency_index", " Gross Domestic Product (GDP)  Normalised"]]
for i in [1,2,3,4,5,6,7,8,9,10]:
    model = VAR(data)
    results = model.fit(i)
    print('Order =', i)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    print()

In [ ]:
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import select_order

data = df[["stringency_index", " Gross Domestic Product (GDP)  Normalised"]]
max_lags = 10
deterministic_terms = 'n'
seasons = 0
exog = None
exog_coint = None
result = select_order(data, max_lags, deterministic=deterministic_terms, seasons=seasons, exog=exog, exog_coint=exog_coint)
selected_lag_order = result.selected_orders['aic']
print(f"Selected lag order based on AIC: {selected_lag_order}")

In [ ]:
# The Null hypothesis for grangercausalitytests is that the time series in the second column,
# x2, does NOT Granger cause the time series in the first column, x1

# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.grangercausalitytests.html#statsmodels.tsa.stattools.grangercausalitytests

# GDP doesn't change the stringency
granger_output = grangercausalitytests(df[["stringency_index", " Gross Domestic Product (GDP)  Normalised"]], maxlag=[selected_lag_order])

In [ ]:
# stringency changes GDP
granger_output = grangercausalitytests(df[[" Gross Domestic Product (GDP)  Normalised", "stringency_index"]], maxlag=[selected_lag_order])